In [1]:
# general functions:
def downsample(time_series,res = '0.2S'):
    # downsamples time_series pandas data frame 
    # time_series: a pandas data.frame for which there is a column 'timestamps' with numeric timestamps
    # res: desired resolution in time after downsampling
    Nvalues = len(time_series.index)
    samplerate = 1/ ((time_series.timestamp[Nvalues-1] - time_series.timestamp[0]) / Nvalues)
    timestart = dt.datetime(1970, 1, 1, 0, 0, 0, 0) #dt.datetime.now()
    start = pd.Timestamp(timestart)
    end = pd.Timestamp(timestart + dt.timedelta(seconds=Nvalues/samplerate))
    t = np.linspace(start.value, end.value, Nvalues)
    t = pd.to_datetime(t)
    time_series['time'] = t
    time_series = time_series.resample(res,on='time').mean() # downsample to 0.2 second intervals
    time_series.index.name = 'time'
    time_series.reset_index(inplace=True)
    return time_series

# Merge features from various modalities into single database

## 1. Load feature data from Video

Load OpenFace output data and load features in pandas dataframe: So far copied from: https://github.com/emrecdem/exploface/blob/master/TUTORIALS/tutorial1.ipynb

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import exploface
import datetime as dt
import numpy as np

In [3]:
exploface.__version__

'0.0.0.dev6'

In [4]:
# specify some paths
emrecdemStudyDataFolder = "/media/sf_sharedfolder/Emotion/emrecdemstudydata"
openface_outputfolder = emrecdemStudyDataFolder + "/OpenFaceOutput"

In [5]:
# Search for files with csv extension, because we are only interested in those
# This assumes that there are no other csv files in folder other than the ones produced by OpenFace.
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

filenames = find_csv_filenames(openface_outputfolder)

In [6]:
filenames

['P18_S2_IAPS_HAPPY_Cfront.csv',
 'P18_S2_IAPS_SAD_Cfront.csv',
 'P21_S2_IAPS_HAPPY_C1.csv',
 'P21_S2_IAPS_SAD_C1.csv']

In [7]:
# Potential loop over filenames, for now just one index
fileindex = 2
deleteDataBaseEntries = False

In [8]:
# select one file to process (in the future this can be a loop over all the files)
openface_file = openface_outputfolder + '/' + filenames[fileindex]
openface_features = exploface.get_feature_time_series(openface_file)

In [9]:
# extract participant id (PID) and experiment id (EXP) from filename:
PID = filenames[fileindex].split("_")[0].split("P")[1]
EXP = ''.join(filenames[fileindex].split("_")[1:4]) # extract integers from filename

In [10]:
EXP = EXP.split(".csv")[0] # remove .csv at the end
print(EXP)

S2IAPSHAPPY


In [11]:
PID

'21'

In [12]:
openface_features = downsample(openface_features,res = '0.2S')

In [13]:
# tidy up data frame:
filter_col = [col for col in openface_features if col.startswith('AU')]
filter_col.insert(0,'time')
filter_col.insert(0,'participant_id')
filter_col.insert(0,'experiment_id')
openface_features['participant_id'] = PID
openface_features['experiment_id'] = EXP
openface_features = openface_features[filter_col]
openface_features.columns = openface_features.columns.str.replace('_', '')
openface_features = openface_features.rename(columns = {'experimentid':'experiment_id'})
openface_features = openface_features.rename(columns = {'participantid':'participant_id'})

In [14]:
openface_features.head()

,experiment_id,participant_id,time,AU01r,AU02r,AU04r,AU05r,AU06r,AU07r,AU09r,...,AU12c,AU14c,AU15c,AU17c,AU20c,AU23c,AU25c,AU26c,AU28c,AU45c
0,S2IAPSHAPPY,21,1970-01-01 00:00:00.000,0.02,0.0,2.496,0.478,1.148,2.074,0.368,...,0.0,0.0,0.0,1.0,0.0,0.8,0.4,0.0,0.2,0.0
1,S2IAPSHAPPY,21,1970-01-01 00:00:00.200,0.00,0.0,2.756,0.538,1.254,1.508,0.346,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,S2IAPSHAPPY,21,1970-01-01 00:00:00.400,0.00,0.0,2.824,0.404,1.368,1.572,0.352,...,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,S2IAPSHAPPY,21,1970-01-01 00:00:00.600,0.00,0.0,2.706,0.626,1.346,1.382,0.332,...,0.0,0.0,0.0,1.0,0.0,0.2,0.0,0.0,0.0,0.0
4,S2IAPSHAPPY,21,1970-01-01 00:00:00.800,0.02,0.0,2.038,0.058,1.450,1.878,0.464,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.8


## 2. Load feature data from audio

Process data with Librosa in pandas dataframe: So far copied from: https://github.com/emrecdem/explibrosa/blob/master/TUTORIALS/tutorial1.ipynb

In [15]:
import matplotlib.pyplot as plt
import os
import explibrosa

In [16]:
explibrosa.__version__

'0.0.0.dev1'

Find wav file that matches the csv file produced by OpenFace based on the assumption that filenames are identical except from file extension

In [17]:
transformfilename = filenames[fileindex]

In [18]:
audiofile_name = transformfilename.replace('.csv','.wav').replace('_Cfront','').replace('_Cside','')

In [19]:
import subprocess
 
# Set up find command
findCMD = 'find ' + emrecdemStudyDataFolder + ' -name ' + audiofile_name 
out = subprocess.Popen(findCMD,shell=True,stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)
# Get standard out and error
(stdout, stderr) = out.communicate()
# Save found files to list
filelist = stdout.decode().split()

In [20]:
audiofiles_fullPaths

NameError: name 'audiofiles_fullPaths' is not defined

In [ ]:
audiofiles_fullPaths = filelist # probably the list has only one filename
audio_file = audiofiles_fullPaths[0]

In [ ]:
os.path.isfile(audio_file)

In [ ]:
explibrosa.get_info(audio_file)

In [ ]:
time_series = explibrosa.get_feature_time_series(audio_file)

In [ ]:
time_series = downsample(time_series,res = '0.2S')

In [ ]:
# Add columns and tidy up column names
time_series['participant_id'] = PID
time_series['experiment_id'] = EXP
time_series = time_series.rename(columns={'zrc': 'zcrate'})
time_series.drop(['timestamp'], axis = 1, inplace = True, errors = 'ignore')
time_series = time_series[['participant_id','experiment_id','time','pitch','rmse','zcrate']]

In [ ]:
time_series.head()

## 3. Store in database


http://www.sqlitetutorial.net/sqlite-python/create-tables/

In [ ]:
import sqlite3

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    # Note time in text datatype, which will facilitate ISO8601 "YYYY-MM-DD HH:MM:SS.SSS"
    database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"
     
    sql_create_participants_table = """ CREATE TABLE IF NOT EXISTS participants (
                                        id integer PRIMARY KEY,
                                        age real
                                    ); """
    
    sql_create_experiments_table = """ CREATE TABLE IF NOT EXISTS experiments (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
    sql_create_videofeatures_table = """ CREATE TABLE IF NOT EXISTS videofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id text NOT NULL,
                                        time text,
                                        AU01r real, AU02r real, AU04r real,
                                        AU05r real, AU06r real, AU07r real,
                                        AU09r real, AU10r real, AU12r real,
                                        AU14r real, AU15r real, AU17r real,
                                        AU20r real, AU23r real, AU25r real,
                                        AU26r real, AU45r real, AU01c real,
                                        AU02c real, AU04c real, AU05c real,
                                        AU06c real, AU07c real, AU09c real,
                                        AU10c real, AU12c real, AU14c real,
                                        AU15c real, AU17c real, AU20c real,
                                        AU23c real, AU25c real, AU26c real,
                                        AU28c real, AU45c real,                                        
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
    
    sql_create_audiofeatures_table = """ CREATE TABLE IF NOT EXISTS audiofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id text NOT NULL,
                                        time text, 
                                        pitch real,
                                        rmse real,
                                        zcrate real,
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
        
    sql_create_facslabels_table = """ CREATE TABLE IF NOT EXISTS facslabels (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
 
    sql_create_facsencoding_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    participant_id integer NOT NULL,
                                    experiment_id integer NOT NULL,
                                    timestamp text,
                                    emotion_id integer NOT NULL,
                                    FOREIGN KEY (participant_id) REFERENCES participants (id),
                                    FOREIGN KEY (experiment_id) REFERENCES experiments (id),
                                    FOREIGN KEY (emotion_id) REFERENCES facslabels (id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create participants table
        create_table(conn, sql_create_participants_table)
        # create experiments table
        create_table(conn, sql_create_experiments_table)
        create_table(conn, sql_create_videofeatures_table)
        create_table(conn, sql_create_audiofeatures_table)
        create_table(conn, sql_create_facslabels_table)
        create_table(conn, sql_create_facsencoding_table)
    else:
        print("Error! cannot create the database connection.")

In [ ]:

if __name__ == '__main__':
    main()

In [ ]:
# Now on command line: sqlite3 pathtomydatabase.db
# should give access to database
# and .tables should show overview of the tables as created

In [ ]:
#https://www.dataquest.io/blog/python-pandas-databases/
database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"

In [ ]:
conn = create_connection(database) # create connection
cur = conn.cursor() # create cursor

### Add Audio features to database

In [ ]:
# delete contents of table for testing purposes, later we obviously do not want to do this
if (deleteDataBaseEntries == True):
    deleterows = cur.execute("DELETE FROM audiofeatures;").fetchall()

In [ ]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [ ]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])

In [ ]:
print("\nColumn names:")
print(colnames)

In [ ]:
# move time_series pandas data frame to the database
time_series.to_sql("audiofeatures", conn, index= False, if_exists="append") #"replace"

In [ ]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [ ]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)
print("\nColumn names:")
print(colnames)

### Add video data to database

In [ ]:
# delete contents of table for testing purposes, later we obviously do not want to do this
if (deleteDataBaseEntries == True):
    deleterows = cur.execute("DELETE FROM videofeatures;").fetchall()

In [ ]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(videofeatures);").fetchall()

In [ ]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nColumn names:")
print(colnames)

In [ ]:
openface_features.head()

In [ ]:
# move time_series pandas data frame to the database
openface_features.to_sql("videofeatures", conn, index=False,if_exists="append") #"replace"

In [ ]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()

In [ ]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)

In [ ]:
cur.close()
conn.close()